In [ ]:
import sys
import pandas as pd

sys.path.append("../")
from scraper.scrapers.lzvcup import LZVCupScraper

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
DIR_COMPETITIONS_TEAMS = "../data/competitions_teams.json"
DIR_STATS = "../data/stats.csv"
# DIR_STATS_HISTORICAL = "../data/stats_historical.csv"

In [ ]:
AREAS = {
    "ANTWERPEN": "results/1",
    "BRUSSELS GEWEST": "results/2",
    "LIMBURG": "results/3",
    "OOST-VLAANDEREN": "results/4",
    "VLAAMS BRABANT": "results/5",
    "WEST-VLAANDEREN": "results/6"
}

In [ ]:
dict_out, list_stats = {}, []

for area_name, area_url in AREAS.items():
    config = {
        "base_url": "https://www.lzvcup.be",
        "area_name": area_name,
        "area_url": area_url
    }

    # initialize scraper instance
    scraper = LZVCupScraper(config)

    # get all regions within area
    region_cards = scraper.parse_region_cards()
    regions = list(region_cards.keys())

    dict_regions = {}
    for region in regions:
        # get all teams for all competitions within a single region
        # note: some competitions appear duplicated across regions (e.g. 4E KLASSE C GENT <> 1E KLASSE DENDERSTREEK, cf. teams Balls & Glory)
        competitions = scraper.parse_competitions_from_region_card(region_cards[region])
        dict_regions[region] = competitions

        # get players and their statistics per team
        df_stats = scraper.parse_teams(competitions["competitions"], region)
        list_stats.append(df_stats)

    dict_out.update({area_name: dict_regions})

# gather DataFrames into one
df_stats_all = pd.concat(list_stats, axis=0)

# store outputs
LZVCupScraper.store_json(dict_out, DIR_COMPETITIONS_TEAMS)
LZVCupScraper.store_csv(df_stats_all, DIR_STATS)

In [ ]:
# LZVCupScraper.parse_player_stats_history(df_stats["Teamleden"].iloc[0], df_stats["_url"].iloc[0])

# df_stats.apply(
#     lambda x: LZVCupScraper.parse_player_stats_history(x.Teamleden, x._url),
#     axis=1
# ).tolist()